# Projet Machine Learning

#### Le but de ce projet est de prédir la température critique d'un superconducteur en utilisant 81 variables.
NB : problème de régression

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# charger les données d'entrainement
data = pd.read_csv("data_train.csv").drop("Unnamed: 0", axis=1)
print("La taille de nos données d'entrainement est ", data.shape)
display(data.head())


# charger les données de test
data_test = pd.read_csv("data_test.csv").drop("Unnamed: 0", axis=1)
print("La taille de nos données de test est ", data_test.shape)
display(data_test.head())

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
#data.describe()
data.info()

## Split des données 

In [ ]:
X = data.drop("critical_temp", axis=1)
y = np.array(data["critical_temp"])
y = y.reshape((y.shape[0],1))

X_test = data_test

print("La taille de notre X_train est", X.shape)
print("La taille de notre y_train est", y.shape)
print("La taille de notre X_test est", X_test.shape)

## Normaliser et standardiser les données

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)
X_test = scaler.transform(X_test)

print(scaler.mean_.shape)

## Appliquer PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99)
pca.fit(X)

# nombre de composantes 
print("On a ", pca.n_components_, "composantes qui nous donne ", pca.explained_variance_ratio_, "de variance \n")

# variance de chaque composante
print("Les variances de chaque composante est :")
print(pca.explained_variance_ratio_, "\n")

# les valeurs propres
print('Les valeurs propres sont :')
print(pca.singular_values_, "\n")

# Appliquer PCA sur les données dentrainement et de test 
X = pca.transform(X)
display("la taille des données après transformation PCA", X.shape)

X_test = pca.transform(X_test)
display("la taille des données de test après transformation PCA", X_test.shape)

plt.scatter(X[:,0], y)

## Machine learning

### 1- Regression linéaire

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error

print(y_val.shape)
reg = LinearRegression()
reg.fit(X_train, y_train)

y_pred = reg.predict(X_val)
print("the mean squared error is : ", mean_squared_error(y_val, y_pred))

print("score sans cross validation")
print(reg.score(X_val, y_val), "\n")

print("Score avec cross validation")
print(cross_val_score(reg, X_train, y_train, cv=5).mean())

plt.scatter(y_val, y_pred, color="black")
plt.plot(y_val, y_val, color='blue', linewidth=3)


Nous remarquons que notre modèle linéaire (regression linéaire n'est pas adapter à nos données)

### 2- Régression polynomiale

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

degree=2
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(X_train,y_train)

y_pred = polyreg.predict(X_val)
print("the mean squared error is : ", mean_squared_error(y_val, y_pred))

print("score sans cross validation")
print(polyreg.score(X_val, y_val))

print("Score avec cross validation")
print(cross_val_score(polyreg, X_train, y_train, cv=10).mean())

plt.scatter(y_val, y_pred, color="black")
plt.plot(y_val, y_val, color='blue', linewidth=3)

In [ ]:

polyregtr = PolynomialFeatures(degree=5)
X_poly = polyregtr.fit_transform(X_train)
polyreg = LinearRegression()
polyreg.fit(X_poly, y_train)
X_val_transformed = polyregtr.transform(X_val)
y_pred = polyreg.predict(X_val_transformed)
print(mean_squared_error(y_val, y_pred))
    
    

Nous remarquons que ce modèle est mieux que le modèle de régression linéaire

### Smooth splines